In [0]:
'''
Iniciando build_aircraft_type_dim
'''

'\nIniciando build_aircraft_type_dim\n'

Importando funções comuns para uso no notebook

In [0]:
%run
./shared_gold_functions

Inicializando sessão spark, definindo o schema da tabela final e importando dados da tabela silver

In [0]:
from pyspark.sql import SparkSession
from pyspark import SparkFiles
from pyspark.sql.types import *
from pyspark.sql import functions as F
import urllib

# Inicializa a sessão Spark
spark = SparkSession.builder.appName("accidents_analysis").getOrCreate()

schema = StructType([
   StructField("aircraft_id", StringType(), False),
   StructField("aircraft_type", StringType(), False),
   StructField("owner_operator", StringType(), True),
   StructField("registration", StringType(), True),
   StructField("msn", StringType(), True),
   StructField("year_of_manufacture", IntegerType(), True),
   StructField("engine_model", StringType(), True),
   StructField("total_airframe_hours", IntegerType(), True),
   StructField("cycles", IntegerType(), True),
   StructField("manufacturer_country_name", StringType(), True),
   StructField("year_of_first_flight_of_model", IntegerType(), True)])

accidents_silver_df = spark.read.format("delta").load('dbfs:/user/hive/warehouse/silver_database.db/silver_accidents')

accidents_silver_df.display()

index,date,time,aircraft_type,owner_operator,registration,msn,year_of_manufacture,engine_model,total_airframe_hrs,cycles,other_fatalities,aircraft_damage,category,approx_location,phase,nature,departure_airport,destination_airport,investigating_agency,confidence_rating,aircraft_type_code,weekday,nb_fatalities,nb_occupants,country_or_region,departure_airport_IATA,departure_airport_ICAO,destination_airport_IATA,destination_airport_ICAO
386881,2024-04-23,1345,Learjet 75,Cimed & Co SA,PP-DYB,45-565,null,null,null,null,0,Substantial,Accident,"Erechim Airport (ERM), RS",Landing,Private,"Chapecó Airport, SC","Erechim-Comandante Kraemer Airport, RS",CENIPA,"Information is only available from news, social media or unofficial sources",LJ75,Tuesday,0,5,Brazil,XAP,SBCH,ERM,SSER
354203,2024-03-10,1452,IAI 1125 Astra SP,SkyJet Elite,N1125A,051,1990,null,null,null,0,Destroyed,Accident,"near Ingalls Field Airport (KHSP), Hot Springs, VA",Approach,Private,"Fort Lauderdale International Airport, FL","Hot Springs-Ingalls Field, VA",NTSB,Information verified through data from accident investigation authorities,ASTR,Sunday,5,5,USA,FLL,KFLL,HSP,KHSP
318650,2023-05-19,1100,Shorts SC.7 Skyvan 3A-100,Uganda Peoples Defence Force,AF-519,SH.1901,1972,null,null,null,0,"Destroyed, written off",Accident,near Kalongo Airstrip,Landing,Military,null,null,null,"Information is only available from news, social media or unofficial sources",SC7,Friday,0,2,Uganda,null,null,null,null
318643,2023-07-08,1500,Antonov An-2R,Aviatörsföreningen Antonov 2,SE-KCE,1G189-59,1981,Shvetsov ASh-62IR,null,null,0,Substantial,Accident,Vårgårda,Take off,Private,null,null,SHK,"Information is only available from news, social media or unofficial sources",AN2,Saturday,0,5,Sweden,null,null,null,null
318648,2023-06-04,1500,Cessna 560 Citation V,Encore Motors of Melbourne Inc,N611VG,560-0091,1990,Pratt & Whitney Canada JT15D-5,null,null,0,"Destroyed, written off",Accident,"near Montebello, VA",En route,Private,null,"Islip-Long Island MacArthur Airport, NY",NTSB,Information verified through data from accident investigation authorities,C560,Sunday,4,4,USA,null,null,ISP,KISP
389190,2024-05-31,null,Airbus A330-203,Air France,F-GZCL,519,2003,GE CF6-80E1A3,null,null,0,Substantial,Accident,N'Djamena Airport (NDJ/FTTJ),Standing,Passenger - Scheduled,N'Djamena Airport,Abuja-Nnamdi Azikiwe International Airport,null,Information verified through data from accident investigation authorities,A332,Friday,0,null,Chad,NDJ,FTTJ,ABV,DNAA
343377,2023-08-09,null,Lockheed CC-130H Hercules,Royal Canadian Air Force (RCAF),130337,382-4584,null,null,null,null,0,Unknown,Accident,"Comox Airport, British Columbia. (YQQ/CYQQ)",Standing,Military,null,null,null,"Information is only available from news, social media or unofficial sources",C130,Wednesday,0,null,Canada,null,null,null,null
351839,2024-02-18,2009,Boeing 767-332ER (WL),Delta Air Lines,N176DZ,29697/745,1999,null,null,null,0,Unknown,Accident,E of Florida,En route,Passenger - Scheduled,"New York-John F. Kennedy International Airport, NY",Bogotá-Eldorado Airport,Aerocivil,"Information is only available from news, social media or unofficial sources",B763,Sunday,0,null,Atlantic Ocean,JFK,KJFK,BOG,SKBO
318674,2023-04-17,null,Antonov An-12BK,Al Quwwat al-Jawwiya As-Sudaniya (Sudanese Air Force),9977,9346302,null,null,null,null,0,"Destroyed, written off",OT,Khartoum International Airport (KRT),Standing,Military,null,null,null,"Information is only available from news, social media or unofficial sources",AN12,Monday,0,0,Sudan,null,null,null,null
344872,2023-08-24,1542,Viking Air DHC-6 Twin Otter 400,Air Antilles,F-OMYS,971,null,null,null,null,0,Substantial,Accident,Saint-Barthélémy-Rémy de Haenen Airport (SBH/TFFJ),Landing,Passenger - Scheduled,Pointe-à-Pitre-Le Raizet Airport,Saint-Barthélémy-Rémy de Haenen Airport,BEA,"Information is only available from news, social media or unofficial sources",DHC6,Thursday,0,6,null,PTP,TFFR,SBH,TFFJ


## Processamento

Na etapa de processamento, vamos:
- Realizar a junção (join) entre as aeronaves envolvidas em acidentes (presente na base silver_accidents) e as informações sobre as aeronaves (presente na base silver_aircrafts) a partir do aircraft_type_code
  - Na tabela silver_aircrafts, temos aeronaves com o mesmo código, mas nomes diferentes, logo precisamos uniformizar e agregar as informações a nível de aircraft_code, de modo que não tenhamos duplicatas de códigos
- Realizar a junção (join) entre as aeronaves e o nome dos países, através do manufacturer_country e do country_code
- Adicionar um código único surrogate (uuid) para cada aeronave envolvida em um acidente (pois apesar de serem modelos diferentes, cada aeronave é única e possui uma história diferente)


Começamos agrupando os dados de aeronaves por aircraft_code, de modo a ter códigos únicos

Como podemos ter nomes diferentes para aeronaves com o mesmo aircraft_code, podemos ter conflitos sobre a informação do ano de primeiro voo da aeronave. Assim, escolhemos pegar o menor valor disponível na base por aircraft_code para fazer essa composição

In [0]:
aircrafts_silver_df = spark.read.format("delta").load('dbfs:/user/hive/warehouse/silver_database.db/silver_aircrafts')

aircrafts_silver_df.exceptAll(aircrafts_silver_df.dropDuplicates(['aircraft_code'])).show()

# Obtendo os registros únicos agregando por aircraft_code e considerando o valor mínimo de year_of_first_flight
aircrafts_silver_df_unique = aircrafts_silver_df.groupBy("aircraft_code").agg(
    F.min(F.when(F.col("year_of_first_flight").isNotNull(), F.col("year_of_first_flight"))).alias("year_of_first_flight"),
    F.first(F.when(F.col("year_of_first_flight").isNotNull(), F.col("aircraft_name")), ignorenulls=True).alias("aircraft_name"),
    F.first(F.col("manufacturer_country"),ignorenulls=True).alias("manufacturer_country")
)

aircrafts_silver_df.display()
aircrafts_silver_df_unique.display()

+--------------------+-------------+--------------------+--------------------+
|       aircraft_name|aircraft_code|year_of_first_flight|manufacturer_country|
+--------------------+-------------+--------------------+--------------------+
|Piper PA-32 Chero...|         PA32|                1963|                   N|
|    Piper PA-32R-300|         P32R|                NULL|                   N|
|             Sling 4|         SLG4|                2011|                  ZS|
|     Embraer EMB-810|         PA34|                NULL|                   N|
|            Snow S-2|         SS2P|                NULL|                   N|
|     Neiva EMB-711ST|         P28T|                NULL|                   N|
|Beechcraft 390 Pr...|         PRM1|                NULL|                   N|
|    Embraer EMB-820C|         PA31|                NULL|                   N|
|       Neiva EMB-711|         P28R|                NULL|                   N|
|Fieseler Fi 156 S...|         F156|                

aircraft_name,aircraft_code,year_of_first_flight,manufacturer_country
Bell 429 GlobalRanger,B429,2007,N
Brumby 610,BR61,2011,VH
Embraer EMB-720,PA32,null,N
Fiat G.212,g212,1947,I
Fournier RF-6,RF6,null,G
Just SuperSTOL,jusu,null,N
Kaman HH-43 Huskie,H43A,1953,N
Taylorcraft Auster 1,PLUS,null,G
Valmet PIK-23 Towmaster,PK23,1982,null
Victa Airtourer,TOUR,1959,VH


aircraft_code,year_of_first_flight,aircraft_name,manufacturer_country
2a2,1917,Salmson 2A2,N
3pln,1916,Sopwith Triplane,G
A1,1945,Douglas A-1 Skyraider,N
A10,1972,Fairchild-Republic A-10 Thunderbolt II,N
A109,1971,Agusta A109,I
A119,1995,Agusta A119 Koala,I
A122,1965,Aerotec Uirapuru,PP
A124,1982,Antonov An-124,UR
A129,1983,Agusta A129 Mangusta,I
A139,2001,AgustaWestland AW139,I


Vamos agora pegar as informações dos países de gold_countries_dim

In [0]:
countries_gold_df = spark.read.format("delta").load('dbfs:/user/hive/warehouse/gold_database.db/gold_countries_dim')

countries_gold_df.display()

country_code,country_name
T2,Tuvalu
YK,Syria
EI,Ireland
F-Oma,Martinique
UK,Uzbekistan
VP-L,British Virgin Islands
TL,Central African Republic
T7,San Marino
OM,Slovakia
TG,Guatemala


Selecionamos as colunas que definem uma aeronave em silver_accidents e realizamos os seguintes passos:
- Adicionamos um uuid
- Realizamos um join com as informações específicas de cada aeronave com base no aircraft_type_code e aircraft_code

In [0]:
aircraft_type_df = accidents_silver_df.select('aircraft_type','aircraft_type_code','owner_operator','registration','msn','year_of_manufacture','engine_model','total_airframe_hrs','cycles')

# Adicionando um uuid para cada aeronave de um acidente
aircraft_type_df = aircraft_type_df.withColumn('aircraft_id', F.expr("uuid()"))

# Dando join com as informações especificas de cada modelo de aeronave
cond = [aircraft_type_df.aircraft_type_code == aircrafts_silver_df_unique.aircraft_code]

aircraft_type_df = aircraft_type_df.join(aircrafts_silver_df_unique,cond,'left')

aircraft_type_df.display()

aircraft_type,aircraft_type_code,owner_operator,registration,msn,year_of_manufacture,engine_model,total_airframe_hrs,cycles,aircraft_id,aircraft_code,year_of_first_flight,aircraft_name,manufacturer_country
Learjet 75,LJ75,Cimed & Co SA,PP-DYB,45-565,null,null,null,null,9e8e8a58-60cc-469d-bf1f-4de64007fe66,LJ75,null,null,N
IAI 1125 Astra SP,ASTR,SkyJet Elite,N1125A,051,1990,null,null,null,d5648ad0-dfa4-44ed-a84f-eaf2ab475c9c,ASTR,1984,IAI 1125 Astra,4X
Shorts SC.7 Skyvan 3A-100,SC7,Uganda Peoples Defence Force,AF-519,SH.1901,1972,null,null,null,463f2afc-b0f1-4993-ae58-02d1ae5d1134,SC7,1963,Shorts SC.7 Skyvan,G
Antonov An-2R,AN2,Aviatörsföreningen Antonov 2,SE-KCE,1G189-59,1981,Shvetsov ASh-62IR,null,null,7d784c5f-aee0-447f-878c-1b55125b60d7,AN2,1947,Antonov An-2,UR
Cessna 560 Citation V,C560,Encore Motors of Melbourne Inc,N611VG,560-0091,1990,Pratt & Whitney Canada JT15D-5,null,null,aec827a3-4665-4889-83bf-099ae82c5cdf,C560,1987,Cessna 560 Citation V,N
Airbus A330-203,A332,Air France,F-GZCL,519,2003,GE CF6-80E1A3,null,null,4f2da891-6bce-4f2c-aa99-001e5d306fd8,A332,1992,Airbus A330-200,EU
Lockheed CC-130H Hercules,C130,Royal Canadian Air Force (RCAF),130337,382-4584,null,null,null,null,8271fa51-79cb-4393-9518-47ef4167f387,C130,1954,Lockheed C-130 & L-100 Hercules,N
Boeing 767-332ER (WL),B763,Delta Air Lines,N176DZ,29697/745,1999,null,null,null,04e2160e-034b-4b97-a4e5-e6cd6bb3a1e7,B763,1986,Boeing 767-300,N
Antonov An-12BK,AN12,Al Quwwat al-Jawwiya As-Sudaniya (Sudanese Air Force),9977,9346302,null,null,null,null,69d9ca9d-302e-49fb-90ca-ef471289bce9,AN12,1957,Antonov An-12,UR
Viking Air DHC-6 Twin Otter 400,DHC6,Air Antilles,F-OMYS,971,null,null,null,null,f22e1efe-d9f3-4eb6-b912-3c734f806173,DHC6,1965,de Havilland Canada DHC-6 Twin Otter,C


Realizamos um join com a tabela de países atraves do manufacturer_country e country_code para obter o nome do país de manufatura

In [0]:
# Adicionando a informação sobre o país de manufatura

cond = [aircraft_type_df.manufacturer_country == countries_gold_df.country_code]

aircraft_type_df = aircraft_type_df.join(countries_gold_df,cond,'left')

aircraft_type_df.display()


aircraft_type,aircraft_type_code,owner_operator,registration,msn,year_of_manufacture,engine_model,total_airframe_hrs,cycles,aircraft_id,aircraft_code,year_of_first_flight,aircraft_name,manufacturer_country,country_code,country_name
Learjet 75,LJ75,Cimed & Co SA,PP-DYB,45-565,null,null,null,null,9e8e8a58-60cc-469d-bf1f-4de64007fe66,LJ75,null,null,N,N,USA
IAI 1125 Astra SP,ASTR,SkyJet Elite,N1125A,051,1990,null,null,null,d5648ad0-dfa4-44ed-a84f-eaf2ab475c9c,ASTR,1984,IAI 1125 Astra,4X,4X,Israel
Shorts SC.7 Skyvan 3A-100,SC7,Uganda Peoples Defence Force,AF-519,SH.1901,1972,null,null,null,463f2afc-b0f1-4993-ae58-02d1ae5d1134,SC7,1963,Shorts SC.7 Skyvan,G,G,United Kingdom
Antonov An-2R,AN2,Aviatörsföreningen Antonov 2,SE-KCE,1G189-59,1981,Shvetsov ASh-62IR,null,null,7d784c5f-aee0-447f-878c-1b55125b60d7,AN2,1947,Antonov An-2,UR,UR,Ukraine
Cessna 560 Citation V,C560,Encore Motors of Melbourne Inc,N611VG,560-0091,1990,Pratt & Whitney Canada JT15D-5,null,null,aec827a3-4665-4889-83bf-099ae82c5cdf,C560,1987,Cessna 560 Citation V,N,N,USA
Airbus A330-203,A332,Air France,F-GZCL,519,2003,GE CF6-80E1A3,null,null,4f2da891-6bce-4f2c-aa99-001e5d306fd8,A332,1992,Airbus A330-200,EU,EU,Europe
Lockheed CC-130H Hercules,C130,Royal Canadian Air Force (RCAF),130337,382-4584,null,null,null,null,8271fa51-79cb-4393-9518-47ef4167f387,C130,1954,Lockheed C-130 & L-100 Hercules,N,N,USA
Boeing 767-332ER (WL),B763,Delta Air Lines,N176DZ,29697/745,1999,null,null,null,04e2160e-034b-4b97-a4e5-e6cd6bb3a1e7,B763,1986,Boeing 767-300,N,N,USA
Antonov An-12BK,AN12,Al Quwwat al-Jawwiya As-Sudaniya (Sudanese Air Force),9977,9346302,null,null,null,null,69d9ca9d-302e-49fb-90ca-ef471289bce9,AN12,1957,Antonov An-12,UR,UR,Ukraine
Viking Air DHC-6 Twin Otter 400,DHC6,Air Antilles,F-OMYS,971,null,null,null,null,f22e1efe-d9f3-4eb6-b912-3c734f806173,DHC6,1965,de Havilland Canada DHC-6 Twin Otter,C,C,Canada


Precisamos verificar se, com essas transformações, alguma aeronave possui data de manufatura anterior ao voo de estreia do modelo (o que seria impossível). Se sim, colocamos o valor do ano do primeiro voo como o ano de manufatura

In [0]:
# Verificando se existe alguma aeronave que possui data de manufatura anterior a data do primeiro voo do modulo. Se sim, colocar a data de manufatura como a data do primeiro voo
aircraft_type_df = aircraft_type_df.withColumn(
        "year_of_manufacture",
        F.when(
            F.col("year_of_manufacture") < F.col("year_of_first_flight"),
            F.col("year_of_first_flight")
        ).otherwise(F.col("year_of_manufacture"))
    )

Finalizadas as transformações, criamos o df com o schema da dimensão

In [0]:
# Criando aircraft_type_dim

aircraft_type_dim = aircraft_type_df.select('aircraft_id','aircraft_type','owner_operator','registration','msn','year_of_manufacture','engine_model','total_airframe_hrs','cycles','country_name','year_of_first_flight')

aircraft_type_dim = spark.createDataFrame(aircraft_type_dim.rdd, schema=schema)

aircraft_type_dim.display()

aircraft_id,aircraft_type,owner_operator,registration,msn,year_of_manufacture,engine_model,total_airframe_hours,cycles,manufacturer_country_name,year_of_first_flight_of_model
9e8e8a58-60cc-469d-bf1f-4de64007fe66,Learjet 75,Cimed & Co SA,PP-DYB,45-565,null,null,null,null,USA,null
d5648ad0-dfa4-44ed-a84f-eaf2ab475c9c,IAI 1125 Astra SP,SkyJet Elite,N1125A,051,1990,null,null,null,Israel,1984
463f2afc-b0f1-4993-ae58-02d1ae5d1134,Shorts SC.7 Skyvan 3A-100,Uganda Peoples Defence Force,AF-519,SH.1901,1972,null,null,null,United Kingdom,1963
7d784c5f-aee0-447f-878c-1b55125b60d7,Antonov An-2R,Aviatörsföreningen Antonov 2,SE-KCE,1G189-59,1981,Shvetsov ASh-62IR,null,null,Ukraine,1947
aec827a3-4665-4889-83bf-099ae82c5cdf,Cessna 560 Citation V,Encore Motors of Melbourne Inc,N611VG,560-0091,1990,Pratt & Whitney Canada JT15D-5,null,null,USA,1987
4f2da891-6bce-4f2c-aa99-001e5d306fd8,Airbus A330-203,Air France,F-GZCL,519,2003,GE CF6-80E1A3,null,null,Europe,1992
8271fa51-79cb-4393-9518-47ef4167f387,Lockheed CC-130H Hercules,Royal Canadian Air Force (RCAF),130337,382-4584,null,null,null,null,USA,1954
04e2160e-034b-4b97-a4e5-e6cd6bb3a1e7,Boeing 767-332ER (WL),Delta Air Lines,N176DZ,29697/745,1999,null,null,null,USA,1986
69d9ca9d-302e-49fb-90ca-ef471289bce9,Antonov An-12BK,Al Quwwat al-Jawwiya As-Sudaniya (Sudanese Air Force),9977,9346302,null,null,null,null,Ukraine,1957
f22e1efe-d9f3-4eb6-b912-3c734f806173,Viking Air DHC-6 Twin Otter 400,Air Antilles,F-OMYS,971,null,null,null,null,Canada,1965


##Avaliando a qualidade dos dados
- Testar se aircraft_id não possui nulo
- Testar se total_airframe_hours é um valor não negativo
- Testar se cycles é um valor não negativo
- Testar se year_of_manufacture é menor ou igual ao ano atual (não podemos ter aeronaves do futuro)
- Testar se year_of_manufacture é maior ou igual a 1903 (fabricação do primeiro avião da história)
- Testar se o year_of_manufacture é maior ou igual ao year_of_first_flight_of_model (não podemos ter um avião faircado antes do ano do primeiro voo do modelo)

In [0]:
# Testando a qualidade dos dados
test_col_not_null(aircraft_type_dim,'aircraft_id')
test_value_range(aircraft_type_dim,"total_airframe_hours","> 0 or total_airframe_hours is null ")
test_value_range(aircraft_type_dim,"cycles","> 0 or cycles is null")

import datetime as dt
current_year = dt.date.today().year
test_value_range(aircraft_type_dim,"year_of_manufacture",f"<= {current_year} or year_of_manufacture is null")
test_value_range(aircraft_type_dim,"year_of_manufacture",">= 1903 or year_of_manufacture is null")
test_value_range(aircraft_type_dim,"year_of_manufacture",">= year_of_first_flight_of_model or year_of_manufacture is null or year_of_first_flight_of_model is null ")

Avaliando a condição aircraft_id não contem nulos
Avaliando a condição total_airframe_hours > 0 or total_airframe_hours is null 
Avaliando a condição cycles > 0 or cycles is null
Avaliando a condição year_of_manufacture <= 2024 or year_of_manufacture is null
Avaliando a condição year_of_manufacture >= 1903 or year_of_manufacture is null
Avaliando a condição year_of_manufacture >= year_of_first_flight_of_model or year_of_manufacture is null or year_of_first_flight_of_model is null 


## Registrando dados no banco
Após aprovação, salvamos o dado sanitizado no banco e verificamos se tivemos sucesso

In [0]:
# Cria o banco de dados se ele não existir
database_name = 'gold_database'
table_name = 'gold_aircraft_type_dim'


spark.sql(f"CREATE DATABASE IF NOT EXISTS {database_name}")

# Salva o DataFrame como tabela Delta
aircraft_type_dim.write.mode("overwrite").format("delta").option("mergeSchema", "true").saveAsTable(
    f"{database_name}.{table_name}"
)

In [0]:
%sql
SELECT * from gold_database.gold_aircraft_type_dim

aircraft_type,owner_operator,registration,msn,year_of_manufacture,engine_model,total_airframe_hours,cycles,manufacturer_country_name,year_of_first_flight_of_model,aircraft_id
Learjet 75,Cimed & Co SA,PP-DYB,45-565,null,null,null,null,USA,null,9e8e8a58-60cc-469d-bf1f-4de64007fe66
IAI 1125 Astra SP,SkyJet Elite,N1125A,051,1990,null,null,null,Israel,1984,d5648ad0-dfa4-44ed-a84f-eaf2ab475c9c
Shorts SC.7 Skyvan 3A-100,Uganda Peoples Defence Force,AF-519,SH.1901,1972,null,null,null,United Kingdom,1963,463f2afc-b0f1-4993-ae58-02d1ae5d1134
Antonov An-2R,Aviatörsföreningen Antonov 2,SE-KCE,1G189-59,1981,Shvetsov ASh-62IR,null,null,Ukraine,1947,7d784c5f-aee0-447f-878c-1b55125b60d7
Cessna 560 Citation V,Encore Motors of Melbourne Inc,N611VG,560-0091,1990,Pratt & Whitney Canada JT15D-5,null,null,USA,1987,aec827a3-4665-4889-83bf-099ae82c5cdf
Airbus A330-203,Air France,F-GZCL,519,2003,GE CF6-80E1A3,null,null,Europe,1992,4f2da891-6bce-4f2c-aa99-001e5d306fd8
Lockheed CC-130H Hercules,Royal Canadian Air Force (RCAF),130337,382-4584,null,null,null,null,USA,1954,8271fa51-79cb-4393-9518-47ef4167f387
Boeing 767-332ER (WL),Delta Air Lines,N176DZ,29697/745,1999,null,null,null,USA,1986,04e2160e-034b-4b97-a4e5-e6cd6bb3a1e7
Antonov An-12BK,Al Quwwat al-Jawwiya As-Sudaniya (Sudanese Air Force),9977,9346302,null,null,null,null,Ukraine,1957,69d9ca9d-302e-49fb-90ca-ef471289bce9
Viking Air DHC-6 Twin Otter 400,Air Antilles,F-OMYS,971,null,null,null,null,Canada,1965,f22e1efe-d9f3-4eb6-b912-3c734f806173


In [0]:
'''
Finalizando build_aircraft_type_dim
'''

'\nFinalizando build_aircraft_type_dim\n'